# Simulating Hess Diagrams with StarFISH

In [1]:
%matplotlib inline
import os
import numpy as np
os.getenv("STARFISH")

'/Users/jsick/code/_starfish/starfish'

## Download isochrones

The [Padova package](http://github.com/jonathansick/padova) lets us quickly download a set of the Padova group's isochrones from the CMD website. Here we'll download a grid of isochrones to cover a set of WFC3 bands. (*Note: PHAT uses ACS for the optical bands*). Padova is able to export these isochrones into a format that StarFISH understands.

In [2]:
isoc_dirname = "padova_isoc"
full_isoc_dirname = os.path.join(os.getenv("STARFISH"),
                                 isoc_dirname)

wfc3_bands = ['F275W', 'F336W', 'F475W', 'F814W', 'F110W', 'F160W']
# z_grid = [0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
#           0.001, 0.0012, 0.0016, 0.002, 0.0025, 0.0031, 0.0039, 0.0049, 0.0061, 0.0077, 0.0096,
#           0.012, 0.015, 0.019, 0.024, 0.03]
z_grid = [0.019]

In [3]:
import time
from padova import AgeGridRequest
for z in z_grid:
    r = AgeGridRequest(z, min_log_age=6.6, max_log_age=10.13, delta_log_age=0.05,
                       phot='wfc3', photsys_version='odfnew')
    for isoc in r.tables.isochrones:
        isoc.export_for_starfish(full_isoc_dirname, bands=wfc3_bands)
    # time.sleep(10)

Reading from cache


In [4]:
from starfisher import LibraryBuilder

input_dir = "wfc3_test"
lib_dir = "padova_lib"
builder = LibraryBuilder(input_dir, isoc_dirname, lib_dir, nmag=len(wfc3_bands))
# builder.install()


## Generating a synthetic Hess CMD library

In [5]:
from starfisher import Lockfile, Synth, ExtinctionDistribution, MockNullCrowdingTable

synth_dir = "wfc3_test/synth"
crowd_path = "wfc3_test/crowding.dat"

# Setup an isochrone locking scheme for 10 age bins
lockfile = Lockfile(builder, synth_dir)
lockfile.lock_grid(np.linspace(6., 10.5, num=10))

# No extinction
young_av = ExtinctionDistribution()
old_av = ExtinctionDistribution()
rel_extinction = np.ones(len(wfc3_bands), dtype=float)
for av in (young_av, old_av):
    av.set_uniform(0.)

# No photometric errors
crowd = MockNullCrowdingTable(crowd_path, len(wfc3_bands))

synth = Synth(input_dir, builder, lockfile, crowd,
              rel_extinction, young_extinction=young_av, old_extinction=old_av)
synth.add_cmd((wfc3_bands.index('F475W'), wfc3_bands.index('F814W')),
              wfc3_bands.index('F814W'),
              (-1., 4.),
              (16., 32.),
              35.,
              'f475f814',
              'F475W-F814W',
              'F814W')
# synth.run_synth(include_unlocked=False)
synth.plot_all_hess('synth_hess', flipy=True)